In [1]:
import os
import math
import numpy as np
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
from datetime import datetime, date, time

batch_size = 64
embedding_dimension = 5
negative_samples = 8

# 텐서보드 모델 요약을 저장할 위치
LOG_DIR = "/home/younseun/columbus_study/04_learning_tensorflow/06__word_embeddings_and_rnns/logs/word2vec_intro_"+datetime.now().strftime('%Y%m%d%H%M%S')
LOG_DIR = "/tmp/logs/word2vec_intro_2"

digit_to_word_map = {1: "One", 2: "Two", 3: "Three", 4: "Four", 5: "Five",
                     6: "Six", 7: "Seven", 8: "Eight", 9: "Nine"}
sentences = []

# Create two kinds of sentences - sequences of odd and even digits.
for i in range(10000):
    rand_odd_ints = np.random.choice(range(1, 10, 2), 3)
    sentences.append(" ".join([digit_to_word_map[r] for r in rand_odd_ints]))
    rand_even_ints = np.random.choice(range(2, 10, 2), 3)
    sentences.append(" ".join([digit_to_word_map[r] for r in rand_even_ints]))

# 하나만 테스트로 진행할때 사용
# sentences = []
# sentences.append("Eight Four Six")

In [2]:
# 단어를 인덱스에 매핑
word2index_map = {}
index = 0
for sent in sentences:
    #print(sent)
    for word in sent.lower().split():
        #print(word)
        if word not in word2index_map:
            word2index_map[word] = index
            index += 1
        #print(word2index_map)

# 역방향 매핑
index2word_map = {index: word for word, index in word2index_map.items()}
vocabulary_size = len(index2word_map)

print('word2index_map :{}'.format(word2index_map))
print('index2word_map :{}'.format(index2word_map))
print('vocabulary_size :{}'.format(vocabulary_size))


word2index_map :{'nine': 0, 'one': 1, 'five': 2, 'two': 3, 'eight': 4, 'three': 5, 'four': 6, 'six': 7, 'seven': 8}
index2word_map :{0: 'nine', 1: 'one', 2: 'five', 3: 'two', 4: 'eight', 5: 'three', 6: 'four', 7: 'six', 8: 'seven'}
vocabulary_size :9


In [3]:
# Generate skip-gram pairs
sample_seq = 0
skip_gram_pairs = []
for sent in sentences:
    tokenized_sent = sent.lower().split()
    sample_seq += 1
    if sample_seq == 1:
        print(tokenized_sent)
    for i in range(1, len(tokenized_sent)-1):
        word_context_pair = [[word2index_map[tokenized_sent[i-1]],
                              word2index_map[tokenized_sent[i+1]]],
                             word2index_map[tokenized_sent[i]]]
        skip_gram_pairs.append([word_context_pair[1],
                                word_context_pair[0][0]])
        skip_gram_pairs.append([word_context_pair[1],
                                word_context_pair[0][1]])
        if sample_seq == 10:
            print(word_context_pair)
            print([word_context_pair[1], word_context_pair[0][0]])
            print([word_context_pair[1], word_context_pair[0][1]])

['nine', 'one', 'five']
[[7, 4], 4]
[4, 7]
[4, 4]


In [4]:
def get_skipgram_batch(batch_size):
    instance_indices = list(range(len(skip_gram_pairs)))
    np.random.shuffle(instance_indices)
    batch = instance_indices[:batch_size]
    x = [skip_gram_pairs[i][0] for i in batch]      # Target word
    y = [[skip_gram_pairs[i][1]] for i in batch]    # 문맥 단어
    return x, y

In [5]:
#skip_gram_pairs 데이터 확인
x, y = get_skipgram_batch(batch_size)
print('x:{}, y:{}'.format(x[:3],y[:3]))
print('x(target word) : ', [index2word_map[index] for index in x[:3]])
print('y(context word) : ', [index2word_map[index[0]] for index in y[:3]])


x:[6, 8, 3], y:[[4], [1], [7]]
x(target word) :  ['four', 'seven', 'two']
y(context word) :  ['eight', 'one', 'six']


In [6]:
# Input data, labels
train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])

In [7]:
# Embedding lookup table currently only implemented in CPU
with tf.name_scope("embeddings"):
    embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_dimension],-1.0, 1.0), name='embedding')
    # This is essentialy a lookup table
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)
embeddings, train_inputs, embed

(<tf.Variable 'embeddings/embedding:0' shape=(9, 5) dtype=float32_ref>,
 <tf.Tensor 'Placeholder:0' shape=(64,) dtype=int32>,
 <tf.Tensor 'embeddings/embedding_lookup:0' shape=(64, 5) dtype=float32>)

In [8]:
# Create variables for the NCE loss
nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_dimension],
                            stddev=1.0 / math.sqrt(embedding_dimension)))
nce_biases = tf.Variable(tf.zeros([vocabulary_size]))


loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weights, biases=nce_biases, inputs=embed, labels=train_labels,
                      num_sampled=negative_samples, num_classes=vocabulary_size))
tf.summary.scalar("NCE_loss", loss)


<tf.Tensor 'NCE_loss_1:0' shape=() dtype=string>

In [9]:
# Learning rate decay
global_step = tf.Variable(0, trainable=False)
learningRate = tf.train.exponential_decay(learning_rate=0.1,
                                          global_step=global_step,
                                          decay_steps=1000,
                                          decay_rate=0.95,
                                          staircase=True)
train_step = tf.train.GradientDescentOptimizer(learningRate).minimize(loss)

merged = tf.summary.merge_all()

In [10]:
first_onoff = 0
with tf.Session() as sess:
    train_writer = tf.summary.FileWriter(LOG_DIR,
                                         graph=tf.get_default_graph())
    saver = tf.train.Saver()

    with open(os.path.join(LOG_DIR, 'metadata.tsv'), "w") as metadata:
        metadata.write('Name\tClass\n')
        for k, v in index2word_map.items():
            metadata.write('%s\t%d\n' % (v, k))

    config = projector.ProjectorConfig()
    embedding = config.embeddings.add()
    embedding.tensor_name = embeddings.name
    # Link this tensor to its metadata file (e.g. labels).
    embedding.metadata_path = os.path.join(LOG_DIR, 'metadata.tsv')
    projector.visualize_embeddings(train_writer, config)

    tf.global_variables_initializer().run()

    for step in range(1000):
        x_batch, y_batch = get_skipgram_batch(batch_size)
        first_onoff += 1
        if first_onoff == 1:
            print(x_batch, y_batch)
        summary, _ = sess.run([merged, train_step],
                              feed_dict={train_inputs: x_batch,
                                         train_labels: y_batch})
        train_writer.add_summary(summary, step)

        if step % 100 == 0:
            saver.save(sess, os.path.join(LOG_DIR, "w2v_model.ckpt"), step)
            loss_value = sess.run(loss,
                                  feed_dict={train_inputs: x_batch,
                                             train_labels: y_batch})
            print("Loss at %d: %.5f" % (step, loss_value))

    # Normalize embeddings before using
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    normalized_embeddings_matrix = sess.run(normalized_embeddings)
print("completed !!")

[5, 3, 2, 6, 6, 3, 3, 1, 0, 8, 7, 5, 3, 3, 8, 5, 4, 0, 3, 5, 3, 7, 5, 2, 8, 7, 3, 1, 1, 3, 7, 0, 1, 7, 2, 3, 2, 7, 5, 7, 6, 0, 1, 8, 4, 3, 3, 2, 5, 8, 0, 7, 4, 0, 7, 0, 8, 8, 4, 3, 6, 3, 8, 6] [[0], [3], [8], [3], [7], [7], [3], [8], [1], [8], [7], [5], [4], [3], [8], [5], [7], [1], [7], [2], [4], [3], [8], [2], [2], [7], [7], [8], [8], [7], [6], [0], [2], [4], [5], [3], [5], [3], [2], [7], [4], [0], [1], [2], [4], [3], [6], [1], [5], [0], [2], [6], [3], [5], [7], [5], [5], [1], [3], [3], [3], [4], [1], [3]]
Loss at 0: 8.49600
Loss at 100: 3.06681
Loss at 200: 2.72572
Loss at 300: 2.64757
Loss at 400: 2.56248
Loss at 500: 2.55670
Loss at 600: 2.52780
Loss at 700: 2.66727
Loss at 800: 2.49300
Loss at 900: 2.62261
Instructions for updating:
keep_dims is deprecated, use keepdims instead
completed !!
